In [2]:
# https://dacon.io/codeshare/2384
# https://catboost.ai/en/docs/concepts/python-reference_catboost_grid_search
# https://www.kaggle.com/code/kevrchen/catboost-gridsearchcv-feature-engineering/notebook

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


import os

In [4]:
train_list = [
    './preprocessed/KNOW_2017_train_bert.csv',
            './preprocessed/KNOW_2017_train_no_text.csv',
            './preprocessed/KNOW_2017_train_simcse.csv',
            

            './preprocessed/KNOW_2018_train_bert.csv',
            './preprocessed/KNOW_2018_train_no_text.csv',
            './preprocessed/KNOW_2018_train_simcse.csv',
            

            './preprocessed/KNOW_2019_train_bert.csv',
            './preprocessed/KNOW_2019_train_no_text.csv',
            './preprocessed/KNOW_2019_train_simcse.csv',
            

            './preprocessed/KNOW_2020_train_bert.csv',
            './preprocessed/KNOW_2020_train_no_text.csv',
            './preprocessed/KNOW_2020_train_simcse.csv',
    './preprocessed/KNOW_2017_train_tfidf.csv',
    './preprocessed/KNOW_2019_train_tfidf.csv',
    './preprocessed/KNOW_2018_train_tfidf.csv',
            './preprocessed/KNOW_2020_train_tfidf.csv']

In [5]:
test_list = [
    './preprocessed/KNOW_2017_test_bert.csv',
            './preprocessed/KNOW_2017_test_no_text.csv',
            './preprocessed/KNOW_2017_test_simcse.csv',
            

            './preprocessed/KNOW_2018_test_bert.csv',
            './preprocessed/KNOW_2018_test_no_text.csv',
            './preprocessed/KNOW_2018_test_simcse.csv',
            

            './preprocessed/KNOW_2019_test_bert.csv',
            './preprocessed/KNOW_2019_test_no_text.csv',
            './preprocessed/KNOW_2019_test_simcse.csv',
            

            './preprocessed/KNOW_2020_test_bert.csv',
            './preprocessed/KNOW_2020_test_no_text.csv',
            './preprocessed/KNOW_2020_test_simcse.csv',
    './preprocessed/KNOW_2017_test_tfidf.csv',
    './preprocessed/KNOW_2018_test_tfidf.csv',
    './preprocessed/KNOW_2019_test_tfidf.csv',
            './preprocessed/KNOW_2020_test_tfidf.csv']

In [6]:
filename_list = [
    './result/KNOW_2017_bert.csv',
            './result/KNOW_2017_no_text.csv',
            './result/KNOW_2017_simcse.csv',
            

            './result/KNOW_2018_bert.csv',
            './result/KNOW_2018_no_text.csv',
            './result/KNOW_2018_simcse.csv',
            

            './result/KNOW_2019_bert.csv',
            './result/KNOW_2019_no_text.csv',
            './result/KNOW_2019_simcse.csv',
            

            './result/KNOW_2020_bert.csv',
            './result/KNOW_2020_no_text.csv',
            './result/KNOW_2020_simcse.csv',
    './result/KNOW_2017_tfidf.csv',
    './result/KNOW_2018_tfidf.csv',
    './result/KNOW_2019_tfidf.csv',
            './result/KNOW_2020_tfidf.csv']

In [7]:
for file_name in train_list:
    if os.path.isfile(file_name):
        print("pass")
    else:
        print(file_name  + " don't exist")

pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass


In [8]:
for file_name in test_list:
    if os.path.isfile(file_name):
        print("pass")
    else:
        print(file_name  + " don't exist")

pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass


In [9]:
print(len(train_list))
print(len(test_list))
print(len(filename_list))

16
16
16


In [10]:
n_split = 5
k_fold = KFold(n_splits = n_split, random_state = 123, shuffle=True)

for i in range(len(filename_list)):

    train = pd.read_csv(train_list[i])
    print(train.dtypes.unique())
    
    
    print(train.dtypes.unique())
    print('\n___' + filename_list[i] + ' start___')

    y = train['knowcode']
    X = train.drop(['index', 'knowcode'], axis=1)

#     y_test = test.copy()['knowcode']
#     X_test = test.drop(['index', 'knowcode'], axis=1)

    with open('./result/result.txt', 'a') as result:
        result.write('___' + filename_list[i] + '___\n')

    catboost_clf = CatBoostClassifier(learning_rate = 0.3, 
                                      iterations = 2200, 
                                      task_type = 'GPU', 
                                      eval_metric='Accuracy',
#                                       l2_leaf_reg=3.16227766e-09,
#                                       loss_function= 'MultiClass',
#                                       leaf_estimation_iterations=10,
                                      random_seed = 123
                                     )

    for j, (train_index, test_index) in enumerate(k_fold.split(train)):
        # Create data for this fold
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
        print( "\nFold ", j)
        with open('./result/result.txt', 'a') as result:
            result.write("Fold: {}".format(j))

        # Run model for this fold

        catboost_clf = catboost_clf.fit( X_train, y_train,
                             cat_features=None,
                             eval_set=(X_valid, y_valid),
                                       )

        # Generate validation predictions for this fold
        pred = catboost_clf.predict(X_valid)

#         print(filename_list[i] + '의 정확도: {0:.4f}'.format(accuracy_score(y_valid , pred)))
        with open('./result/result.txt', 'a') as result:
            result.write('정확도: {0:.4f}\n'.format(accuracy_score(y_valid , pred)))
    
    test = pd.read_csv(test_list[i])
    pred_y = catboost_clf.predict(test[train.drop(['index','knowcode'], axis=1).columns])
    
    submission = pd.DataFrame({
        'index': test['index'],
        'knowcode': pred_y.transpose()[0]
    })
    submission.to_csv(filename_list[i], index=False)

[dtype('int64') dtype('float64')]
[dtype('int64') dtype('float64')]

___./result/KNOW_2017_bert.csv start___

Fold  0


In [24]:
# 애러날경우 수동으로
# pred_y = catboost_clf.predict(test.drop(['index'], axis=1))
pred_y = catboost_clf.predict(test[train.drop(['knowcode', 'index'], axis=1).columns])
pred_y

array([[904201],
       [215105],
       [411201],
       ...,
       [561301],
       [155401],
       [122103]], dtype=int64)

In [10]:
submission = pd.DataFrame({
    'index': test['index'],
    'knowcode': pred_y.transpose()[0]
})
submission0

,index,knowcode
0,0,904201
1,1,415501
2,2,411103
3,3,411103
4,4,23402
...,...,...
9064,9064,531401
9065,9065,122201
9066,9066,122201
9067,9067,122201


In [25]:
submission.to_csv(filename_list[0], index=False)

In [2]:
# tf-idf만 다시
train_list = [
    './preprocessed/KNOW_2017_train_tfidf.csv',
    './preprocessed/KNOW_2019_train_tfidf.csv',
    './preprocessed/KNOW_2018_train_tfidf.csv',
            './preprocessed/KNOW_2020_train_tfidf.csv']

In [3]:
test_list = [
    './preprocessed/KNOW_2017_test_tfidf.csv',
    './preprocessed/KNOW_2018_test_tfidf.csv',
    './preprocessed/KNOW_2019_test_tfidf.csv',
            './preprocessed/KNOW_2020_test_tfidf.csv']

In [4]:
filename_list = [
    './result/KNOW_2017_tfidf.csv',
    './result/KNOW_2018_tfidf.csv',
    './result/KNOW_2019_tfidf.csv',
            './result/KNOW_2020_tfidf.csv']

In [5]:
train = pd.read_csv(train_list[0], dtype='float32')
#     test = pd.read_csv(test_list[0])

In [6]:
train.head()

,index,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq38_1_tfidf_926,bq38_1_tfidf_927,bq38_1_tfidf_928,bq38_1_tfidf_929,bq38_1_tfidf_930,bq38_1_tfidf_931,bq38_1_tfidf_932,bq38_1_tfidf_933,bq38_1_tfidf_934,bq38_1_tfidf_935
0,0.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,4.0,5.0,4.0,5.0,3.0,4.0,3.0,4.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,3.0,4.0,3.0,4.0,3.0,4.0,5.0,6.0,4.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,3.0,3.0,3.0,3.0,3.0,5.0,4.0,5.0,4.0,...,0.0,0.0,0.0,0.660679,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,4.0,5.0,3.0,4.0,3.0,4.0,4.0,5.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train.dtypes

index               float32
aq1_1               float32
aq1_2               float32
aq2_1               float32
aq2_2               float32
                     ...   
bq38_1_tfidf_931    float32
bq38_1_tfidf_932    float32
bq38_1_tfidf_933    float32
bq38_1_tfidf_934    float32
bq38_1_tfidf_935    float32
Length: 24437, dtype: object

In [8]:
train.dtypes == np.float64

index               False
aq1_1               False
aq1_2               False
aq2_1               False
aq2_2               False
                    ...  
bq38_1_tfidf_931    False
bq38_1_tfidf_932    False
bq38_1_tfidf_933    False
bq38_1_tfidf_934    False
bq38_1_tfidf_935    False
Length: 24437, dtype: bool

In [9]:
np.where(train.dtypes == np.float64)

(array([], dtype=int64),)

In [10]:
train.dtypes.unique()

array([dtype('float32')], dtype=object)

In [5]:
# 계속 커널이 죽는다

n_split = 5
k_fold = KFold(n_splits = n_split, random_state = 123, shuffle=True)

for i in range(len(filename_list)):

#     train = pd.read_csv(train_list[i], dtype='float16')
    train = pd.read_csv(train_list[i])
    
    print('___' + filename_list[i] + ' start___')

    y = train['knowcode']
    X = train.drop(['index', 'knowcode'], axis=1)

#     y_test = test.copy()['knowcode']
#     X_test = test.drop(['index', 'knowcode'], axis=1)

    with open('./result/result.txt', 'a') as result:
        result.write('___' + filename_list[i] + '___\n')

    catboost_clf = CatBoostClassifier(learning_rate = 0.3, 
                                      iterations = 2200, 
                                      task_type = 'GPU', 
                                      eval_metric='Accuracy',
#                                       l2_leaf_reg=3.16227766e-09,
#                                       loss_function= 'MultiClass',
#                                       leaf_estimation_iterations=10,
                                      random_seed = 123
                                     )

    for j, (train_index, test_index) in enumerate(k_fold.split(train)):
        # Create data for this fold
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
        print( "\nFold ", j)
        with open('./result/result.txt', 'a') as result:
            result.write("Fold: {}\n".format(j))


        # Run model for this fold

        catboost_clf = catboost_clf.fit( X_train, y_train,
                             cat_features=None,
                             eval_set=(X_valid, y_valid),
                                       )

        # Generate validation predictions for this fold
        pred = catboost_clf.predict(X_valid)

#         print(filename_list[i] + '의 정확도: {0:.4f}'.format(accuracy_score(y_valid , pred)))
        with open('./result/result.txt', 'a') as result:
            result.write(filename_list[i] + '의 정확도: {0:.4f}\n'.format(accuracy_score(y_valid , pred)))
            
    test = pd.read_csv(test_list[i])
    pred_y = catboost_clf.predict(test[train.drop(['index','knowcode'], axis=1).columns])

    submission = pd.DataFrame({
        'index': test['index'],
        'knowcode': pred_y.transpose()[0]
    })
    submission.to_csv(filename_list[i], index=False)

___./result/KNOW_2017_tfidf.csv start___

Fold  0


TypeError: No matching signature found